In [1]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.datasets import cifar10

import numpy as np

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, num_classes=10)

170500096/170498071 [==============================] - 3s 0us/step


In [3]:
print('x_train shape: {} | y_train shape: {}\nx_test shape : {} | y_test shape : {}'.format(
    x_train.shape, y_train.shape,x_test.shape, y_test.shape))

x_train shape: (50000, 32, 32, 3) | y_train shape: (50000, 10)
x_test shape : (10000, 32, 32, 3) | y_test shape : (10000, 1)


In [4]:
input_shape = x_train[0,:,:,:].shape
model_input = Input(shape=input_shape)

## First model: ConvPool-CNN-C

In [5]:
def conv_pool_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [16]:
conv_pool_cnn_model = conv_pool_cnn(model_input)

In [17]:
def compile_and_train(model, num_epochs): 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    history = model.fit(x=x_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, validation_split=0.2)
    return model

In [18]:
model01 = compile_and_train(conv_pool_cnn_model, num_epochs=10)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 58s 1ms/step - loss: 2.3028 - acc: 0.0995 - val_loss: 2.3028 - val_acc: 0.0980
Epoch 2/10
40000/40000 [==============================] - 57s 1ms/step - loss: 2.3028 - acc: 0.0993 - val_loss: 2.3028 - val_acc: 0.0952
Epoch 3/10
40000/40000 [==============================] - 57s 1ms/step - loss: 2.3028 - acc: 0.0965 - val_loss: 2.3028 - val_acc: 0.0952
Epoch 4/10
40000/40000 [==============================] - 57s 1ms/step - loss: 2.3028 - acc: 0.0983 - val_loss: 2.3028 - val_acc: 0.0952
Epoch 5/10
40000/40000 [==============================] - 57s 1ms/step - loss: 2.3028 - acc: 0.0977 - val_loss: 2.3027 - val_acc: 0.1014
Epoch 6/10
40000/40000 [==============================] - 57s 1ms/step - loss: 2.3028 - acc: 0.0995 - val_loss: 2.3028 - val_acc: 0.0952
Epoch 7/10
40000/40000 [==============================] - 57s 1ms/step - loss: 2.3028 - acc: 0.1008 - val_loss: 2.3027 - val_acc

In [19]:
def evaluate_error(model):
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]    
    return error

In [20]:
evaluate_error(conv_pool_cnn_model)

0.9

## Second model: ALL-CNN-C

In [21]:
def all_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='all_cnn')
    
    return model

In [22]:
all_cnn_model = all_cnn(model_input)

In [23]:
model02 = compile_and_train(all_cnn_model, num_epochs=10)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 55s 1ms/step - loss: 1.8100 - acc: 0.3086 - val_loss: 1.5588 - val_acc: 0.4049
Epoch 2/10
40000/40000 [==============================] - 54s 1ms/step - loss: 1.3487 - acc: 0.5019 - val_loss: 1.1818 - val_acc: 0.5643
Epoch 3/10
40000/40000 [==============================] - 54s 1ms/step - loss: 1.0658 - acc: 0.6142 - val_loss: 0.9526 - val_acc: 0.6585
Epoch 4/10
40000/40000 [==============================] - 54s 1ms/step - loss: 0.8837 - acc: 0.6854 - val_loss: 0.8499 - val_acc: 0.7049
Epoch 5/10
40000/40000 [==============================] - 54s 1ms/step - loss: 0.7520 - acc: 0.7319 - val_loss: 0.7843 - val_acc: 0.7279
Epoch 6/10
40000/40000 [==============================] - 54s 1ms/step - loss: 0.6394 - acc: 0.7745 - val_loss: 0.7728 - val_acc: 0.7321
Epoch 7/10
40000/40000 [==============================] - 54s 1ms/step - loss: 0.5441 - acc: 0.8082 - val_loss: 0.7604 - val_acc

In [24]:
evaluate_error(all_cnn_model)

0.2491

## Third Model: Network In Network CNN

In [25]:
def nin_cnn(model_input):
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model

In [26]:
nin_cnn_model = nin_cnn(model_input)

In [27]:
model03 = compile_and_train(nin_cnn_model, num_epochs=10)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 17s 415us/step - loss: 1.8804 - acc: 0.2838 - val_loss: 1.6992 - val_acc: 0.3696
Epoch 2/10
40000/40000 [==============================] - 15s 387us/step - loss: 1.5874 - acc: 0.4088 - val_loss: 1.4558 - val_acc: 0.4677
Epoch 3/10
40000/40000 [==============================] - 16s 388us/step - loss: 1.4683 - acc: 0.4587 - val_loss: 1.4791 - val_acc: 0.4570
Epoch 4/10
40000/40000 [==============================] - 16s 388us/step - loss: 1.3852 - acc: 0.4951 - val_loss: 1.3127 - val_acc: 0.5279
Epoch 5/10
40000/40000 [==============================] - 16s 388us/step - loss: 1.3206 - acc: 0.5203 - val_loss: 1.2754 - val_acc: 0.5482
Epoch 6/10
40000/40000 [==============================] - 15s 387us/step - loss: 1.2611 - acc: 0.5488 - val_loss: 1.1526 - val_acc: 0.5875
Epoch 7/10
40000/40000 [==============================] - 15s 387us/step - loss: 1.2151 - acc: 0.5654 - val_loss: 1.

In [28]:
evaluate_error(nin_cnn_model)

0.3611

## Ensemble

In [29]:
models = []
models.append(model01)
models.append(model02)
models.append(model03)

In [30]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [31]:
ensemble_model = ensemble(models, model_input)

In [32]:
evaluate_error(ensemble_model)

0.2379